In [9]:
# Import necessary libraries
import json
import os

import pandas as pd
import openai
import sys
# Append the parent folder to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
# Now you can import config.py
from groq import Groq
from dotenv import load_dotenv
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm

In [6]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

False

In [1]:
import pandas as pd

df = pd.read_parquet("hf://datasets/lavita/ChatDoctor-iCliniq/data/train-00000-of-00001-7f15f39e4c3a7ee9.parquet")

In [2]:
df.shape

(7321, 4)

In [4]:
df.columns

Index(['input', 'answer_icliniq', 'answer_chatgpt', 'answer_chatdoctor'], dtype='object')

In [5]:
df.head(3)

,input,answer_icliniq,answer_chatgpt,answer_chatdoctor
0,"Hello doctor,I had mumps five months ago and a...","Hello, Welcome to Chat Doctor forum. I can und...",It is possible that you still have an infectio...,1) There is no need to worry about reinfection...
1,"Hello doctor,I am trying to conceive but my hu...","Hello, Wellcome to ChatDoctor. There are few r...",Hello! Thank you for reaching out.\n\nCocaine ...,1) Cocaine use can affect fertility by causing...
2,"Hello doctor,I am currently on the Yaz contrac...",Hello. The brown discharge should be temporary...,Hello! It's not uncommon to experience some ir...,1) The brown discharge is due to the hormonal ...


# Embeddings columns

In [6]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np

In [7]:
# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')
# Generate embeddings for the input column
question_embeddings = model.encode(df['input'].tolist(), convert_to_numpy=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nakka\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

C:\Users\nakka\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
df.head()

,input,answer_icliniq,answer_chatgpt,answer_chatdoctor
0,"Hello doctor,I had mumps five months ago and a...","Hello, Welcome to Chat Doctor forum. I can und...",It is possible that you still have an infectio...,1) There is no need to worry about reinfection...
1,"Hello doctor,I am trying to conceive but my hu...","Hello, Wellcome to ChatDoctor. There are few r...",Hello! Thank you for reaching out.\n\nCocaine ...,1) Cocaine use can affect fertility by causing...
2,"Hello doctor,I am currently on the Yaz contrac...",Hello. The brown discharge should be temporary...,Hello! It's not uncommon to experience some ir...,1) The brown discharge is due to the hormonal ...
3,"Hello doctor,My son aged 13 years has got his ...","Hello, Welcome to Chat Doctor forum. I have go...","As an AI language model, I am not a doctor and...","Hi, welcome to Chat Doctor forum. I understand..."
4,"Hello doctor,I just got one side of my wisdom ...",Hello. The lump is mostly a hard swelling whic...,"Hello,\n\nIt is normal to have some swelling a...","Hi, Thanks for your query. It appears that you..."


In [9]:
df.columns

Index(['input', 'answer_icliniq', 'answer_chatgpt', 'answer_chatdoctor'], dtype='object')

In [11]:
question_embeddings

array([[-0.03187011, -0.01305723,  0.03453286, ..., -0.05491394,
         0.06469719,  0.04996957],
       [-0.00565126, -0.00903827, -0.04345672, ...,  0.06669873,
         0.03208437, -0.03703978],
       [-0.00982077, -0.04181525, -0.05341623, ..., -0.16608874,
        -0.02976253,  0.06784043],
       ...,
       [ 0.05819432, -0.03614795, -0.03024961, ..., -0.01062565,
        -0.02319748,  0.1076974 ],
       [-0.02549623, -0.05261699, -0.05192121, ..., -0.10069826,
         0.01680332, -0.03787581],
       [ 0.04309299, -0.06010891, -0.01793998, ..., -0.03323974,
         0.03040628,  0.00521657]], dtype=float32)

# Create a FAISS Index and Add Embeddings:

In [12]:
# Define the dimensionality of the embeddings
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add the embeddings to the FAISS index
index.add(np.array(question_embeddings, dtype='float32'))


In [36]:
query = "I am trying to conceive, but my husband uses cocaine. What should I do?"

query = "Can i exercise with pain in my knee"


query_embedding = model.encode([query], convert_to_numpy=True)

# Search for the nearest neighbors
k = 6  # Number of closest matches
distances, indices = index.search(np.array(query_embedding, dtype='float32'), k)

# Retrieve the results
results = df.iloc[indices[0]]
print(results)

matching_questions = df.iloc[indices[0]]  # Get the row data of closest matches

                                                  input  \
4017  Hello doctor,I have pain in the right knee for...   
5569  Hello doctor,I used to play sports. Now after ...   
406   Hi doctor,I started with the gym last week. I ...   
3727  Hello doctor,I am 21 years old, medically fit....   
3293  Hello doctor,I have knee pain for the past one...   
2044  Hello doctor,I am 26 years old, and I weigh 96...   

                                         answer_icliniq  \
4017  Hello, Welcome to Chat Doctor forum. Your X-ra...   
5569  Hello, Welcome to Chat Doctor forum. I read yo...   
406   Hi. It is common to have pain when you start a...   
3727  Hi. I have thoroughly gone through your case a...   
3293  Hello, Welcome to Chat Doctor forum. Pain may ...   
2044  Hello, Welcome to Chat Doctor forum. You are v...   

                                         answer_chatgpt  \
4017  Hello,\n\nBased on your description, it seems ...   
5569  Hello! It's important to rest and allow your m..

In [40]:
matching_questions.head()

,input,answer_icliniq,answer_chatgpt,answer_chatdoctor
4017,"Hello doctor,I have pain in the right knee for...","Hello, Welcome to Chat Doctor forum. Your X-ra...","Hello,\n\nBased on your description, it seems ...","Hi, Welcome to Chat Doctor.com I am Chat Docto..."
5569,"Hello doctor,I used to play sports. Now after ...","Hello, Welcome to Chat Doctor forum. I read yo...",Hello! It's important to rest and allow your m...,1) First - rule out any injury to your knee jo...
406,"Hi doctor,I started with the gym last week. I ...",Hi. It is common to have pain when you start a...,I'm sorry to hear about your pain. Knee pain i...,1) Pain in knee after exercise - this is due t...
3727,"Hello doctor,I am 21 years old, medically fit....",Hi. I have thoroughly gone through your case a...,"Hi! Based on the symptoms you have described, ...","Hi, Welcome to Chat Doctor.com I am Chat Docto..."
3293,"Hello doctor,I have knee pain for the past one...","Hello, Welcome to Chat Doctor forum. Pain may ...","Hello,\n\nThank you for getting in touch. Knee...","Hi, thank you for providing the brief history ..."


## Prepare the Answer Retrieval

In [28]:
from groq import Groq
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
# Setup the OpenAI client to use either Groq, OpenAI.com, or Ollama API
import os
load_dotenv(override=True)
API_HOST = os.getenv("API_HOST")
API_HOST

'groq'